In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import load_model
from urllib.request import urlopen
from PIL import Image
import io

In [2]:
# Existing preprocessing configurations and mappings
age_group_map = {
    '18-29': 0,
    '30-39': 1,
    '40-49': 2,
    '50-59': 3,
    'more than 60': 4
}

# Reverse age group map for decoding predictions
reverse_age_group_map = {v: k for k, v in age_group_map.items()}

MODEL_MAP = {
    1: {
        'name': 'efficientnet_v2',
        'preprocess': tf.keras.applications.efficientnet_v2.preprocess_input,
        'input_size': (224, 224)
    },
    2: {
        'name': 'resnet50',
        'preprocess': tf.keras.applications.resnet50.preprocess_input,
        'input_size': (224, 224)
    },
    3: {
        'name': 'inception_v3',
        'preprocess': tf.keras.applications.inception_v3.preprocess_input,
        'input_size': (299, 299)
    },
    4: {
        'name': 'mobilenet_v2',
        'preprocess': tf.keras.applications.mobilenet_v2.preprocess_input,
        'input_size': (224, 224)
    }
}


In [3]:
def select_model(model_path,selected_model_number):

    model_info = MODEL_MAP[selected_model_number]
    model_name = model_info['name']
    preprocess_func = model_info['preprocess']
    target_size = model_info['input_size']

    print(f"Loading model '{model_name}' from {model_path}...")
    model = load_model(model_path)
    print(f"Model '{model_name}' loaded successfully.")

    return model, preprocess_func, target_size

def load_image(image_source, target_size):

    try:
        if image_source.startswith('http://') or image_source.startswith('https://'):
            print(f"Loading image from URL: {image_source}")
            response = urlopen(image_source)
            image_data = response.read()
            image = Image.open(io.BytesIO(image_data)).convert('RGB')
        else:
            print(f"Loading image from path: {image_source}")
            image = Image.open(image_source).convert('RGB')
    except Exception as e:
        raise ValueError(f"Error loading image from {image_source}: {e}")

    image = image.resize(target_size)
    image_array = np.array(image)
    return image_array

def predict_image(model, preprocess_func, target_size, image_source):

    # Load and preprocess the image
    image = load_image(image_source, target_size)
    image = preprocess_func(image)
    image = np.expand_dims(image, axis=0)  # Add batch dimension

    # Run prediction
    predictions = model.predict(image)

    # Handle different model output structures
    if isinstance(predictions, list):
        if len(predictions) < 3:
            raise ValueError("Model should have three outputs: face_output, age_output, gender_output.")
        face_pred, age_pred, gender_pred = predictions[:3]
    elif isinstance(predictions, dict):
        face_pred = predictions.get('face_output')
        age_pred = predictions.get('age_output')
        gender_pred = predictions.get('gender_output')
        if face_pred is None or age_pred is None or gender_pred is None:
            raise ValueError("Model outputs should include 'face_output', 'age_output', and 'gender_output'.")
    else:
        raise ValueError("Unsupported model output format. Expected list or dict.")

    # Process face prediction
    face_prob = float(face_pred[0][0]) 
    face_label = 1 if face_prob >= 0.5 else 0  

    result = {'face': 'Face' if face_label == 1 else 'Non-Face'}

    if face_label == 1:
        # Process gender prediction
        gender_prob = float(gender_pred[0][0])  
        gender_label = 1 if gender_prob >= 0.5 else 0  # 1: Male, 0: Female
        gender = 'Male' if gender_label == 1 else 'Female'
        result['gender'] = gender

        # Process age prediction
        age_probs = age_pred[0]  
        age_label = int(np.argmax(age_probs))
        age_group = reverse_age_group_map.get(age_label, 'Unknown')
        result['age'] = age_group

    return result


In [4]:
selected_model_number = 4
model_path = '../results/Multi_Task/MobileNetV2_MultiTask/MobileNetV2_MultiTask.h5'
model, preprocess_func, target_size = select_model(model_path,selected_model_number)

# Predict on a new image
image_source = 'https://cdn.prod.www.spiegel.de/images/49f97d1c-5277-477c-8040-bf78adaaa83f_w1024_r1.778_fpx48_fpy33.jpg'
prediction = predict_image(model, preprocess_func, target_size, image_source)
print(prediction)

Loading model 'mobilenet_v2' from ../results/Multi_Task/MobileNetV2_MultiTask/MobileNetV2_MultiTask.h5...
Model 'mobilenet_v2' loaded successfully.
Loading image from URL: https://cdn.prod.www.spiegel.de/images/49f97d1c-5277-477c-8040-bf78adaaa83f_w1024_r1.778_fpx48_fpy33.jpg
1/1 [==============================] - 5s 5s/step
{'face': 'Face', 'gender': 'Male', 'age': 'more than 60'}


In [5]:
image_source='https://media.wired.com/photos/6643f935a984e349e5fda5a6/1:1/w_1600,h_1600,c_limit/Ilya-Sutskever-Leaves-OpenAI-Business-1258459705.jpg'

In [6]:
prediction = predict_image(model, preprocess_func, target_size, image_source)
print(prediction)

Loading image from URL: https://media.wired.com/photos/6643f935a984e349e5fda5a6/1:1/w_1600,h_1600,c_limit/Ilya-Sutskever-Leaves-OpenAI-Business-1258459705.jpg
1/1 [==============================] - 0s 30ms/step
{'face': 'Face', 'gender': 'Male', 'age': '50-59'}


In [7]:
image_source='https://i.guim.co.uk/img/media/8649348d194259744b0d75240c711cb5d221de70/13_72_2386_1431/master/2386.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=d730f53ab1cc888740837e21c57ea047'

In [8]:
prediction = predict_image(model, preprocess_func, target_size, image_source)
print(prediction)

Loading image from URL: https://i.guim.co.uk/img/media/8649348d194259744b0d75240c711cb5d221de70/13_72_2386_1431/master/2386.jpg?width=1200&height=1200&quality=85&auto=format&fit=crop&s=d730f53ab1cc888740837e21c57ea047
1/1 [==============================] - 0s 32ms/step
{'face': 'Face', 'gender': 'Female', 'age': '30-39'}


In [9]:
image_source = 'https://www.rheinland-pilgern.de/data/ort/8/koelner-dom.8-1-5yduj.wide.jpg'

In [10]:
prediction = predict_image(model, preprocess_func, target_size, image_source)
print(prediction)

Loading image from URL: https://www.rheinland-pilgern.de/data/ort/8/koelner-dom.8-1-5yduj.wide.jpg
1/1 [==============================] - 0s 29ms/step
{'face': 'Non-Face'}
